### 지도 불러오기 위해 설치해야 하는 패키지 
- %pip install geopandas
- %pip install folium

In [15]:
import geopandas as gpd
import pandas as pd

# 1. 서울 행정구역 GeoJSON 데이터 불러오기 (인터넷에서 자동으로)
url = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'
seoul_df = gpd.read_file(url)

# 2. 컬럼명 한글로 변경
seoul_df = seoul_df.rename(columns={
    'name': '법정동',
    'code': '법정동코드'
})

# 3. 샘플 데이터 추가 (거래 가격, 거래 수량)
import numpy as np
seoul_df['거래 가격'] = np.random.randint(50000, 400000, len(seoul_df))
seoul_df['거래수량'] = np.random.randint(10, 500, len(seoul_df))

In [16]:
seoul_df.head()

,법정동코드,법정동,name_eng,base_year,geometry,거래 가격,거래수량
0,11250,강동구,Gangdong-gu,2013,"POLYGON ((127.1152 37.55753, 127.1188 37.55722...",238331,74
1,11240,송파구,Songpa-gu,2013,"POLYGON ((127.06907 37.52228, 127.07496 37.520...",264693,111
2,11230,강남구,Gangnam-gu,2013,"POLYGON ((127.05867 37.5263, 127.06907 37.5222...",100114,293
3,11220,서초구,Seocho-gu,2013,"POLYGON ((127.01397 37.52504, 127.01918 37.520...",279709,99
4,11210,관악구,Gwanak-gu,2013,"POLYGON ((126.98368 37.47386, 126.98464 37.469...",339313,248


In [17]:
from folium import Icon, Marker
import folium 
import pandas as pd

# 데이터 불러오기
subway_df = pd.read_csv('data/subway_feature.csv')
bus_df = pd.read_csv('data/bus_feature.csv')

print(f"지하철역: {len(subway_df)}개")
print(f"버스 정류장: {len(bus_df)}개")

# 서울 지도 생성
seoul_map = folium.Map(
    location=[37.541, 126.986], 
    zoom_start=11,
    min_zoom=10,
    max_zoom=16
)

# 지하철역 표시 - 빨강색
for idx, row in subway_df.iterrows():
    if pd.notna(row['위도']) and pd.notna(row['경도']):
        folium.CircleMarker(
            location=[row['위도'], row['경도']],
            radius=5,  # 크기
            popup=f"(S) {row['역사명']}<br>{row['호선']}",
            tooltip=f"(S) {row['역사명']}",
            color='red',  # 테두리 색
            fill=True,
            fillColor='red',  # 채우기 색
            fillOpacity=0.7,
            weight=1
        ).add_to(seoul_map)

# 버스 정류장 표시 - 초록색
# 버스 정류장이 너무 많으니까 샘플링 (100개마다 1개만)
bus_sample = bus_df[::100]  # 너무 많으면 지도가 무거워져요!

for idx, row in bus_sample.iterrows():
    # X좌표=경도, Y좌표=위도
    if pd.notna(row['Y좌표']) and pd.notna(row['X좌표']):
        folium.CircleMarker(
            location=[row['Y좌표'], row['X좌표']],
            radius=3,  # 버스는 조금 작게
            popup=f"(B) {row['정류소명']}<br>{row['정류소 타입']}",
            tooltip=f"(B) {row['정류소명']}",
            color='green',
            fill=True,
            fillColor='green',
            fillOpacity=0.6,
            weight=1
        ).add_to(seoul_map)

# 지도 표시
seoul_map

지하철역: 768개
버스 정류장: 12584개


In [18]:
# 구별 평균 가격, 거래수량 시각화 
folium.Choropleth( 
    geo_data=seoul_df, 
    data=seoul_df, 
    columns=['법정동', '거래 가격'], 
    key_on="feature.properties.법정동", 
    fill_color='YlGnBu', 
    fill_opacity=0.8, 
    line_opacity=0.6, 
    line_color='#1B9C85', 
    legend_name="KRW", 
    smooth_factor=0, 
    Highlight=True, 
    name="평균 아파트 가격(만원)", 
    show=False, 
    overlay=True, 
    nan_fill_color="white",
    threshold_scale=[0, 60000, 110000, 160000, 210000, 260000, 310000, 360000, 410000]
).add_to(seoul_map)

style_function = lambda x: { 
    'fillColor': '#FFFFFF', 
    'color': '#000000', 
    'fillOpacity': 0.1, 
    'weight': 0.1
} 

highlight_function = lambda x: { 
    'fillColor': '#000000', 
    'color': '#000000', 
    'fillOpacity': 0.50, 
    'weight': 0.1
} 

feature = folium.features.GeoJson( 
    data=seoul_df, 
    style_function=style_function, 
    control=False, 
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip( 
        fields=['법정동', '거래수량'], 
        aliases=['지역', '거래수량'], 
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    ) 
) 
seoul_map.add_child(feature) 
seoul_map.keep_in_front(feature)

# 모든 작업이 끝난 후 마지막에 출력!
seoul_map